In [236]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as signal

In [238]:
#  TODO: Add functionality to handle joining multiple csvs together from different file locations into usable df

In [330]:
#  Example df with correct columns

path = "C:/Users/Eric/Downloads/emg_dataframe.csv"
df = pd.read_csv(path, header=None)
df.columns = df.iloc[0]
df = df.drop(0, axis=0).reset_index(drop=True)

rand_data = np.random.rand(1000, len(df.columns))
df_rand = pd.DataFrame(rand_data, columns=df.columns)

df = df.dropna(axis=1, how='all')
df_rand = df_rand.dropna(axis=1, how='all')

# Concatenate the dataframes
df = pd.concat([df, df_rand], ignore_index=True)

In [332]:
df.columns

Index(['time 0', 'sensor 0 EMG 1 (1259.259 Hz)', 'sensor 0 IMP 1 (74.074 Hz)',
       'sensor 0 ACC X (148.148 Hz)', 'sensor 0 ACC Y (148.148 Hz)',
       'sensor 0 ACC Z (148.148 Hz)', 'sensor 0 GYRO X (148.148 Hz)',
       'sensor 0 GYRO Y (148.148 Hz)', 'sensor 0 GYRO Z (148.148 Hz)',
       'time 1', 'sensor 1 EMG 1 (1259.259 Hz)', 'sensor 1 IMP 1 (74.074 Hz)',
       'sensor 1 ACC X (148.148 Hz)', 'sensor 1 ACC Y (148.148 Hz)',
       'sensor 1 ACC Z (148.148 Hz)', 'sensor 1 GYRO X (148.148 Hz)',
       'sensor 1 GYRO Y (148.148 Hz)', 'sensor 1 GYRO Z (148.148 Hz)',
       'time 2', 'sensor 2 EMG 1 (1259.259 Hz)', 'sensor 2 IMP 1 (74.074 Hz)',
       'sensor 2 ACC X (148.148 Hz)', 'sensor 2 ACC Y (148.148 Hz)',
       'sensor 2 ACC Z (148.148 Hz)', 'sensor 2 GYRO X (148.148 Hz)',
       'sensor 2 GYRO Y (148.148 Hz)', 'sensor 2 GYRO Z (148.148 Hz)',
       'time 3', 'sensor 3 EMG 1 (1259.259 Hz)', 'sensor 3 IMP 1 (74.074 Hz)',
       'sensor 3 ACC X (148.148 Hz)', 'sensor 3 ACC Y 

In [334]:
#just for reference rn
data = {
    'row': [0],
    'time': [1, 10, 19, 28, 37],
    'emg': [2, 11, 20, 29, 38],
    'imp': [3, 12, 21, 30, 39],
    'accx': [4, 13, 22, 31, 40],
    'accy': [5, 14, 23, 32, 41],
    'accz': [6, 15, 24, 33, 42],
    'gyrx': [7, 16, 25, 34, 43],
    'gyry': [8, 17, 26, 35, 44],
    'gyrz': [9, 18, 27, 36, 45],
}
start_row = 110
end_row_emg = 396800
end_row_other = 46700

In [336]:
#  Drop unnecessary columns
columns_to_drop = [col for col in df.columns if ('IMP' in col.upper() or 'TIME' in col.upper()) and col != 'time 0']
df = df.drop(columns=columns_to_drop)

In [338]:
emg_columns = ['sensor 0 EMG 1 (1259.259 Hz)',
               'sensor 1 EMG 1 (1259.259 Hz)',
               'sensor 2 EMG 1 (1259.259 Hz)',
               'sensor 3 EMG 1 (1259.259 Hz)',
               'sensor 4 EMG 1 (1259.259 Hz)',
               'sensor 5 EMG 1 (1259.259 Hz)']

In [340]:
#  Resample data to a new frequency.

def resample_data(df, target_fs):
    """
    Resample data to the target sampling frequency.
    
    Parameters:
    - df: The DataFrame containing the time and signal data.
    - target_fs: The target sampling frequency (Hz).
    
    Returns:
    - df_resampled: A new DataFrame with the resampled data.
    """

    time_column = df['time 0']
    emg_columns = [col for col in df.columns if 'EMG' in col]
    gyro_accel_columns = [col for col in df.columns if 'GYRO' in col or 'ACC' in col]
    
    def resample_signal(signal_data, original_fs, target_fs, target_length):
        resampled_signal = signal.resample(signal_data, target_length)
        return resampled_signal
    
    #  Calculate the new time column for resampled data
    original_time_interval = np.mean(np.diff(time_column))
    total_duration = time_column.iloc[-1] - time_column.iloc[0]
    new_time = np.linspace(time_column.iloc[0], time_column.iloc[0] + total_duration, int(total_duration * target_fs))
    
    df_resampled = pd.DataFrame({'time 0': new_time})
    
    for col in emg_columns:
        target_length = len(new_time)
        df_resampled[col] = resample_signal(df[col].values, 1259.259, target_fs, target_length)
    
    for col in gyro_accel_columns:
        target_length = len(new_time)
        df_resampled[col] = resample_signal(df[col].values, 148.148, target_fs, target_length)
    
    return df_resampled

In [342]:
#  Implement butterworth filters of different orders, cutoff frequencies, sampling frequencies, and filter_band type to columns of a dataframe.

def butterworth_filter(data, order, cutoff, fs, filter_band='low'):
    nyquist = 0.5 * fs
    normalized_cutoff = np.array(cutoff) / nyquist
    
    #  Create Butterworth filter coefficients
    if filter_band == 'low':
        b, a = signal.butter(order, normalized_cutoff[0], btype='low')
    elif filter_band == 'high':
        b, a = signal.butter(order, normalized_cutoff[0], btype='high')
    elif filter_band == 'bandpass':
        b, a = signal.butter(order, normalized_cutoff, btype='bandpass')
    elif filter_band == 'bandstop':
        b, a = signal.butter(order, normalized_cutoff, btype='bandstop')
    else:
        raise ValueError(f"unsupported filter: {filter_band}")
    
    #  Apply filter
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

def apply_filter(df, columns, order=4, cutoff=[1.0], fs=1000, filter_band='low'):
    '''
    Apply a Butterworth filter of specified order and cutoff to the given columns of the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the filter to (e.g., EMG columns).
    - order: The order of the filter.
    - cutoff: The cutoff frequency or frequencies.
    - fs: Sampling frequency of the signal.
    - filter_band: Type of filter ('low', 'high', 'bandpass', 'bandstop').
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    for col in columns:
        df[col] = butterworth_filter(df[col].values, order, cutoff, fs, filter_band)
    
    return df

In [344]:
def apply_notch(df, columns, notch_freq=60, fs=1000, Q=30):
    '''
    Apply a notch filter to remove a specific frequency (e.g., powerline noise at 60Hz)
    to the specified columns in the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the notch filter to.
    - notch_freq: Frequency to remove with the notch filter (e.g., 60Hz for powerline noise).
    - fs: Sampling frequency of the signal.
    - Q: Quality factor for the notch filter.
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    nyquist = 0.5 * fs
    normalized_frequency = notch_freq / nyquist

    b, a = signal.iirnotch(normalized_frequency, Q)

    for col in columns:
        df[col] = signal.filtfilt(b, a, df[col].values)
    
    return df

In [346]:
#  TODO: Linear interpolation for nulls (idk if we have nulls)

In [348]:
#  Scale data with minmax
columns_non_time = df.columns[1:]

scaler_minmax = MinMaxScaler()
df[columns_non_time] = scaler_minmax.fit_transform(df[columns_non_time])

#  Apply butterworth filter
df = apply_filter(df, emg_columns)

#  Apply notch filter
df = apply_notch(df, emg_columns)

df

,time 0,sensor 0 EMG 1 (1259.259 Hz),sensor 0 ACC X (148.148 Hz),sensor 0 ACC Y (148.148 Hz),sensor 0 ACC Z (148.148 Hz),sensor 0 GYRO X (148.148 Hz),sensor 0 GYRO Y (148.148 Hz),sensor 0 GYRO Z (148.148 Hz),sensor 1 EMG 1 (1259.259 Hz),sensor 1 ACC X (148.148 Hz),...,sensor 4 GYRO X (148.148 Hz),sensor 4 GYRO Y (148.148 Hz),sensor 4 GYRO Z (148.148 Hz),sensor 5 EMG 1 (1259.259 Hz),sensor 5 ACC X (148.148 Hz),sensor 5 ACC Y (148.148 Hz),sensor 5 ACC Z (148.148 Hz),sensor 5 GYRO X (148.148 Hz),sensor 5 GYRO Y (148.148 Hz),sensor 5 GYRO Z (148.148 Hz)
0,0.306369,0.501439,0.583973,0.453760,0.352510,0.427443,0.825693,0.211492,0.234996,0.834384,...,0.175209,0.023578,0.244932,0.208192,0.177556,0.268574,0.741273,0.406327,0.363240,0.694020
1,0.964745,0.501359,0.973440,0.159645,0.525174,0.196610,0.077445,0.246047,0.236063,0.838845,...,0.397100,0.014832,0.392383,0.209304,0.155754,0.376407,0.093323,0.002425,0.131510,0.029078
2,0.014856,0.501279,0.749933,0.644995,0.508135,0.279544,0.332546,0.962665,0.237135,0.559790,...,0.451558,0.641513,0.739210,0.210421,0.470074,0.892245,0.515443,0.684055,0.318913,0.071800
3,0.305451,0.501199,0.968619,0.668893,0.245207,0.308593,0.499670,0.799465,0.238209,0.936136,...,0.561626,0.565996,0.966004,0.211542,0.040918,0.291881,0.795516,0.943261,0.117741,0.975133
4,0.442176,0.501119,0.414577,0.135935,0.483938,0.995112,0.558441,0.659562,0.239285,0.727184,...,0.570625,0.839670,0.014583,0.212663,0.879326,0.489102,0.765535,0.428424,0.703567,0.837200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.611310,0.489348,0.868717,0.122297,0.459916,0.328581,0.679600,0.997866,0.547470,0.734235,...,0.773154,0.083158,0.457488,0.546399,0.898331,0.239744,0.177519,0.195786,0.352962,0.440638
996,0.804096,0.489348,0.615352,0.626462,0.036271,0.601738,0.797249,0.408579,0.547470,0.340717,...,0.095572,0.793368,0.656483,0.546399,0.403355,0.088665,0.746727,0.211602,0.082020,0.116860
997,0.185312,0.489348,0.336419,0.941536,0.075261,0.139171,0.048398,0.505571,0.547470,0.968056,...,0.045796,0.673956,0.335535,0.546399,0.486683,0.814432,0.519604,0.207168,0.488145,0.930198
998,0.278968,0.489348,0.633616,0.000000,0.491870,0.513949,0.385776,0.127956,0.547470,0.246411,...,0.507291,0.003240,0.767457,0.546398,0.268766,0.014993,0.618886,0.133222,0.404754,0.977615


In [350]:
#  TODO: Sliding window with adjustable window size

In [352]:
#  TODO: Reshape data to fit LSTM

In [354]:
#  TODO: Train/test split

In [356]:
#  TODO: Data augmentation ?? 

In [358]:
#  TODO: Evaluate performance of different filters

In [364]:
#  More descriptive column names for plotting and such

sensor_muscles = {
    0: 'Gastrocnemius', 
    1: 'Biceps Femoris', 
    2: 'Quadriceps Femoris', 
    3: 'Gastrocnemius', 
    4: 'Biceps Femoris', 
    5: 'Quadriceps Femoris'
}

column_mapping = {}

for sensor_num in range(6):
    for signal_type in ['EMG 1', 'ACC X', 'ACC Y', 'ACC Z', 'GYRO X', 'GYRO Y', 'GYRO Z']:
        old_name = f'sensor {sensor_num} {signal_type} (148.148 Hz)' if signal_type != 'EMG 1' else f'sensor {sensor_num} {signal_type} (1259.259 Hz)'
        new_name = f"{sensor_muscles[sensor_num]} {signal_type}"
        column_mapping[old_name] = new_name
column_mapping['time 0'] = 'Time'

df.rename(columns=column_mapping, inplace=True)

In [366]:
df

,Time,Gastrocnemius EMG 1,Gastrocnemius ACC X,Gastrocnemius ACC Y,Gastrocnemius ACC Z,Gastrocnemius GYRO X,Gastrocnemius GYRO Y,Gastrocnemius GYRO Z,Biceps Femoris EMG 1,Biceps Femoris ACC X,...,Biceps Femoris GYRO X,Biceps Femoris GYRO Y,Biceps Femoris GYRO Z,Quadriceps Femoris EMG 1,Quadriceps Femoris ACC X,Quadriceps Femoris ACC Y,Quadriceps Femoris ACC Z,Quadriceps Femoris GYRO X,Quadriceps Femoris GYRO Y,Quadriceps Femoris GYRO Z
0,0.306369,0.501439,0.583973,0.453760,0.352510,0.427443,0.825693,0.211492,0.234996,0.834384,...,0.175209,0.023578,0.244932,0.208192,0.177556,0.268574,0.741273,0.406327,0.363240,0.694020
1,0.964745,0.501359,0.973440,0.159645,0.525174,0.196610,0.077445,0.246047,0.236063,0.838845,...,0.397100,0.014832,0.392383,0.209304,0.155754,0.376407,0.093323,0.002425,0.131510,0.029078
2,0.014856,0.501279,0.749933,0.644995,0.508135,0.279544,0.332546,0.962665,0.237135,0.559790,...,0.451558,0.641513,0.739210,0.210421,0.470074,0.892245,0.515443,0.684055,0.318913,0.071800
3,0.305451,0.501199,0.968619,0.668893,0.245207,0.308593,0.499670,0.799465,0.238209,0.936136,...,0.561626,0.565996,0.966004,0.211542,0.040918,0.291881,0.795516,0.943261,0.117741,0.975133
4,0.442176,0.501119,0.414577,0.135935,0.483938,0.995112,0.558441,0.659562,0.239285,0.727184,...,0.570625,0.839670,0.014583,0.212663,0.879326,0.489102,0.765535,0.428424,0.703567,0.837200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.611310,0.489348,0.868717,0.122297,0.459916,0.328581,0.679600,0.997866,0.547470,0.734235,...,0.773154,0.083158,0.457488,0.546399,0.898331,0.239744,0.177519,0.195786,0.352962,0.440638
996,0.804096,0.489348,0.615352,0.626462,0.036271,0.601738,0.797249,0.408579,0.547470,0.340717,...,0.095572,0.793368,0.656483,0.546399,0.403355,0.088665,0.746727,0.211602,0.082020,0.116860
997,0.185312,0.489348,0.336419,0.941536,0.075261,0.139171,0.048398,0.505571,0.547470,0.968056,...,0.045796,0.673956,0.335535,0.546399,0.486683,0.814432,0.519604,0.207168,0.488145,0.930198
998,0.278968,0.489348,0.633616,0.000000,0.491870,0.513949,0.385776,0.127956,0.547470,0.246411,...,0.507291,0.003240,0.767457,0.546398,0.268766,0.014993,0.618886,0.133222,0.404754,0.977615
